In [53]:
# Vitaly 
import requests
from bs4 import BeautifulSoup
import re

immo_data = {
    "10657599": {},
    "10655844": {},
    "10656608": {},
    "10654023": {},
    "10655537": {},
    "10647497": {},
    "10642679": {},
    "10132944": {},
    "10648643": {},
    "10657189": {},
    "10649460": {},
    "10643043": {},
    "10649446": {},
    "10456986": {}
}
required_info = [
    "type", "subtype", 
    "country", "region", "province", "district", "locality", "postalCode",
    "street", "number", "box",
    "price"
]

def extractJSdata(str):
    data = dict()
    for s in required_info:
        match = re.search(r"\"" + re.escape(s) + "\":\"?([a-zA-Z 0-9,:]*)\"?", str)
        if match != None:
            value = match.group(1)
        else:
            value = "N/A"
        data[s] = value
    return data

data_categories = []

def scrape_property_info():
    session = requests.Session()

    for key, value in immo_data.items():
        url = "https://www.immoweb.be/en/classified/" + key
        immo_data[key]["URL"] = url

        req = session.get(url)
        status = req.status_code

        if status != 200:
            print(status)
            print(url)
            immo_data[key]["Status"] = status
        else:
            immo_data[key]["Status"] = status
            content = req.content
            s = BeautifulSoup(content, "html.parser")

            js_data = dict()

            ### to scrape from JS
            script_tags = s.find_all('script', {'type': 'text/javascript'})
            for st in script_tags:
                if st.text.find("window.classified") != -1:
                    js_var = re.search(r"\"property\":{.*\};", st.text)
                    js_text = js_var.group(0)
                    js_data = extractJSdata(js_text)
                    break
                else:
                    continue

            html_data = dict()
            for tag in s.find_all("th"):
                dataType = tag.contents[0].strip()
                dataValue = tag.findNext("td").contents[0].strip()
                if dataValue == "":
                    ### Scrape from children
                    dataValue = "???"
                html_data[dataType] = dataValue
                if dataType not in data_categories:
                    data_categories.append(dataType)

            immo_data[key]["Data"] = js_data
            immo_data[key]["Extras"] = html_data

scrape_property_info()

with open("test.json", "w", encoding="utf-8") as json_file:
    json_file.write(json.dumps(immo_data, indent=4))

data_categories.sort()
print(len(data_categories))
for cat in data_categories:
    print(cat)

97

Accessible for disabled people
Address
Agent's name
Air conditioning
Armored door
As built plan
Attic
Available as of
Available date
Basement
Basement surface
Bathrooms
Bedroom 1 surface
Bedroom 2 surface
Bedroom 3 surface
Bedroom 4 surface
Bedroom 5 surface
Bedrooms
Building condition
CO₂ emission
Cadastral income
Caretaker
Common water heater
Conformity certification for fuel tanks
Connection to sewer network
Construction year
Covered parking spaces
Dining room
Double glazing
Dressing room
Elevator
Energy class
External reference
Flat land
Flood zone type
Floor
Furnished
Garden
Garden orientation
Garden surface
Gas, water & electricity
Heat pump
Heating type
How many fireplaces?
Intercom
Internet
Isolated
Jacuzzi
Kitchen surface
Kitchen type
Land is facing street
Latest land use designation
Laundry room
Living area
Living room
Living room surface
Monthly charges
Neighbourhood or locality
Number of floors
Number of frontages
Obligation to build
Office
Office surface
Outdoor parkin